<style>
@import url('https://fonts.googleapis.com/css2?family=Fira+Code:wght@300..700&display=swap');
</style>

<div style="text-align: center; font-family: 'Fira Code', serif;">
        
import libraries

</div>

In [1]:
import torch
import torch.nn as nn
from torchvision.datasets import MNIST

from torch.optim import Adam
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from torchvision.datasets import MNIST
from tqdm.notebook import trange
from torch.optim.lr_scheduler import LambdaLR

import matplotlib.pyplot as plt
import cv2
import gdown
import os

from model import Model, count_parameters, load_model
from part_module import cosine_schedule, forward_process

<style>
@import url('https://fonts.googleapis.com/css2?family=Fira+Code:wght@300..700&display=swap');
</style>

<div style="text-align: center; font-family: 'Fira Code', serif;">
        
init var - download file

</div>

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu')

In [ ]:
# file_url = "https://drive.google.com/uc?id=1NhUbtDHdDKCch_ss-zNVLjSM1n9LDORk&confirm=t"
# out_put = "checkpoint.pth"

# if os.path.exists(f"./{out_put}") == False:
#     gdown.download(file_url, out_put, quiet=False)
#     print("Download successfully !!!")
# else:
#     print("file existed")

<style>
@import url('https://fonts.googleapis.com/css2?family=Fira+Code:wght@300..700&display=swap');
</style>

<div style="text-align: center; font-family: 'Fira Code', serif;">
        
build model

</div>

In [6]:
model = Model(image_channels=1).to(device=device)
count_parameters(model)

<style>
@import url('https://fonts.googleapis.com/css2?family=Fira+Code:wght@300..700&display=swap');
</style>

<div style="text-align: center; font-family: 'Fira Code', serif;">
        
load dataset

</div>

In [7]:
batch_size =  32

dataset = MNIST('.', train=True, transform=transforms.ToTensor(), download=True)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=4)

<style>
@import url('https://fonts.googleapis.com/css2?family=Fira+Code:wght@300..700&display=swap');
</style>

<div style="text-align: center; font-family: 'Fira Code', serif;">
        
build optim - loss - ...

</div>

In [8]:
lr=1e-4

optimizer = Adam(model.parameters(), lr=lr)
scheduler = LambdaLR(optimizer, lr_lambda=lambda epoch: max(0.2, 0.98 ** epoch))
mse_loss_fn = nn.MSELoss()
t = torch.randint(0, 1000, (batch_size,), device=device).float()

In [ ]:
# url = "./checkpoint.pth"
# check = False

# load_model(url, model, optimizer, check)

<style>
@import url('https://fonts.googleapis.com/css2?family=Fira+Code:wght@300..700&display=swap');
</style>

<div style="text-align: center; font-family: 'Fira Code', serif;">
        
train our model

</div>

In [ ]:
timesteps = 1000
n_epochs = 100
tqdm_epoch = trange(n_epochs)

model.train()

for epoch in tqdm_epoch:
    for x, y in dataloader:
        x = x.to(device)
        y = x.clone()

        steps = torch.randint(0, timesteps, (x.shape[0], ), device=device)
        x0 = forward_process(x, timesteps, cosine_schedule, x.shape[0], device)
        assert not torch.isnan(x0).any(), "x0 contains NaN!"
        assert not torch.isinf(x0).any(), "x0 contains Inf!"

        y_pred = model(x0, steps)
        loss = mse_loss_fn(y_pred, y)
        optimizer.zero_grad()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()

    scheduler.step()
    lr_current = scheduler.get_last_lr()[0]

    if torch.isnan(loss) or torch.isinf(loss):
        print("Loss contains NaN or Inf! Stopping training.")
        break

    print(f"epoch --> {epoch+1}")
    print("loss value: {:5f}, learning rate: {:.1e}".format(loss.item(), lr_current))
    tqdm_epoch.set_description('Loss: {:5f}'.format(loss.item()))

    del loss, x0, y_pred
    torch.cuda.empty_cache()

In [ ]:
torch.save({
    'epoch': epoch,
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict()
}, "checkpoint_1.pth")

<style>
@import url('https://fonts.googleapis.com/css2?family=Fira+Code:wght@300..700&display=swap');
</style>

<div style="text-align: center; font-family: 'Fira Code', serif;">
        
put model into eval and plot the prediction

</div>

In [7]:
img = cv2.imread('cat.png')
img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
img = cv2.resize(img, (128, 128))
img = torch.tensor(img/255., dtype=torch.float32, device=device)
img = img.unsqueeze(0).unsqueeze(0)

In [14]:
timesteps = 1000
img_0 = forward_process(img, timesteps, cosine_schedule, img.shape[0], device)
t = torch.randint(0, timesteps, (img_0.shape[0], ), device=device)

In [16]:
model.eval()
y_pred = model(img_0, t)

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(10, 5))

ax[0].imshow(img.squeeze().detach().cpu().numpy(), cmap='gray')
ax[0].set_title("Real Image")

ax[1].imshow(img_0.squeeze().detach().cpu(), cmap='gray')
ax[1].set_title("Noisy Image")

ax[2].imshow(y_pred.squeeze().detach().cpu(), cmap='gray')
ax[2].set_title("Reconstructed Image")


plt.tight_layout()
plt.show()